In [1]:
import pickle
import math
import numpy as np
import time
from collections import defaultdict

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [2]:
AutoML_path = ""
log_folder = "alphad3m_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_MF = AutoML_path+log_folder+'/meta_features_02272022'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022'

combination_name = 'MF+OneHot'
today = '03102022'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_'+today

In [3]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

MF = pickle.load(open(log_MF, "rb"))
MF_keys = list(MF[list(MF.keys())[0]][0].keys())
MF_np = {}
for task in MF:
    feature = [MF[task][0][key]['value'] for key in MF_keys]
    feature = [0 if type(i) != float or math.isnan(i) else i for i in feature]
    MF_np[task] = np.array(feature)
print(len(MF_np))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))



39
39
7811


In [4]:
target_test_pred_time = {}
data_names = list(MF_np.keys())
data_embed = MF_np

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        prev_key = dict_key
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len(task_train) == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        X_train = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_train])
        y_train = np.array([i[3] for i in pipeline_steps_class if i[0] in task_train])
        X_test = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_test])
        y_test = np.array([i[3] for i in pipeline_steps_class if i[0] in task_test])
        print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

        model = MLPRegressor(hidden_layer_sizes=(256,128,64), learning_rate='adaptive', random_state=19)
        model.fit(X_train, y_train)
        end = time.time()
        duration = end-start

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print("Threshold", threshold, "MAE:", mae, "MSE:", mse)
        print("Time:", duration)
        target_test_pred_time[dict_key] = (y_test, y_pred, duration, mae, mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
X_train shape: (1526, 267) y_train shape: (1526,)
X_test shape: (452, 267) y_test shape: (452,)
Threshold 0.05 MAE: 1.4029568360693723 MSE: 1.986963007026519
Time: 1.2659649848937988
X_train shape: (3405, 267) y_train shape: (3405,)
X_test shape: (452, 267) y_test shape: (452,)
Threshold 0.1 MAE: 0.4653627290613527 MSE: 0.2305532673588938
Time: 3.7968320846557617
X_train shape: (5415, 267) y_train shape: (5415,)
X_test shape: (452, 267) y_test shape: (452,)
Threshold 0.15000000000000002 MAE: 0.2741713924919571 MSE: 0.09121558682527997
Time: 9.304840087890625
X_train shape: (5892, 267) y_train shape: (5892,)
X_test shape: (452, 267) y_test shape: (452,)
Threshold 0.2 MAE: 0.1300536751177112 MSE: 0.03166069384606729
Time: 8.742568016052246
X_train shape: (6745, 267) y_train shape: (6745,)
X_test shape: (452, 267) y_test shape: (452,)
Threshold 0.25 MAE: 0.20890795198266493 MSE: 0.06095229278260843
Time: 8.561959266662598
X_train shape: (6931, 267) y_train shape: (6931,)
X_test sh

Threshold 0.1 MAE: 0.33369891470461516 MSE: 0.11781151821772114
Time: 4.325986862182617
X_train shape: (5708, 267) y_train shape: (5708,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.18832409358144814 MSE: 0.04318411448036275
Time: 6.964535236358643
X_train shape: (6128, 267) y_train shape: (6128,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.2 MAE: 0.23384290566212038 MSE: 0.06214644168543647
Time: 6.425379753112793
X_train shape: (6981, 267) y_train shape: (6981,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.25 MAE: 0.1571938261526572 MSE: 0.031078825263009008
Time: 8.32919192314148
X_train shape: (7167, 267) y_train shape: (7167,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.3 MAE: 0.1348060739020527 MSE: 0.020870180002904016
Time: 8.996937990188599
Australian_MF+OneHot_0.3 Australian_MF+OneHot_0.35
35 35
X_train shape: (7383, 267) y_train shape: (7383,)
X_test shape: (216, 267) y_test shape: (216,)
Thresho

Threshold 0.25 MAE: 3.980920037543764 MSE: 15.873099018878733
Time: 5.9576451778411865
X_train shape: (6785, 267) y_train shape: (6785,)
X_test shape: (196, 267) y_test shape: (196,)
Threshold 0.3 MAE: 1.8269299534104864 MSE: 3.361730566360939
Time: 3.8181190490722656
X_train shape: (7006, 267) y_train shape: (7006,)
X_test shape: (196, 267) y_test shape: (196,)
Threshold 0.35000000000000003 MAE: 1.0136516683753378 MSE: 1.0510207898572144
Time: 6.989048957824707
X_train shape: (7187, 267) y_train shape: (7187,)
X_test shape: (196, 267) y_test shape: (196,)
Threshold 0.4 MAE: 2.325192195426155 MSE: 5.434050195094944
Time: 8.194797039031982
X_train shape: (7403, 267) y_train shape: (7403,)
X_test shape: (196, 267) y_test shape: (196,)
Threshold 0.45 MAE: 3.663761919143496 MSE: 13.449111830031878
Time: 9.466753959655762
11 connect-4
X_train shape: (1694, 267) y_train shape: (1694,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.05 MAE: 0.19489913128960887 MSE: 0.041690502673156

Threshold 0.3 MAE: 0.6782979002464063 MSE: 0.47214637390068837
Time: 3.7721691131591797
X_train shape: (7403, 267) y_train shape: (7403,)
X_test shape: (204, 267) y_test shape: (204,)
Threshold 0.35000000000000003 MAE: 0.6137986489722881 MSE: 0.38962604359623404
Time: 6.791013717651367
dionis_MF+OneHot_0.35 dionis_MF+OneHot_0.4
37 37
X_train shape: (7607, 267) y_train shape: (7607,)
X_test shape: (204, 267) y_test shape: (204,)
Threshold 0.45 MAE: 0.42583972630283173 MSE: 0.1949873839821273
Time: 5.850048065185547
16 fabert
X_train shape: (665, 267) y_train shape: (665,)
X_test shape: (188, 267) y_test shape: (188,)
Threshold 0.05 MAE: 0.48363151502604024 MSE: 0.24266396569755652
Time: 0.9403929710388184
X_train shape: (1919, 267) y_train shape: (1919,)
X_test shape: (188, 267) y_test shape: (188,)
Threshold 0.1 MAE: 0.11334365376269026 MSE: 0.01896900395274623
Time: 2.2531070709228516
X_train shape: (2977, 267) y_train shape: (2977,)
X_test shape: (188, 267) y_test shape: (188,)
Thres

Threshold 0.15000000000000002 MAE: 0.3896046827515228 MSE: 0.1533804574941713
Time: 1.9649569988250732
jannis_MF+OneHot_0.15 jannis_MF+OneHot_0.2
15 15
X_train shape: (4935, 267) y_train shape: (4935,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.25 MAE: 0.23752412365574502 MSE: 0.05755150977497963
Time: 5.3610382080078125
X_train shape: (5756, 267) y_train shape: (5756,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.3 MAE: 0.1506110779956878 MSE: 0.02410951261391804
Time: 4.49442720413208
X_train shape: (6884, 267) y_train shape: (6884,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.35000000000000003 MAE: 0.37657991318078743 MSE: 0.14381682357091888
Time: 4.342402935028076
X_train shape: (7595, 267) y_train shape: (7595,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.4 MAE: 0.3055504653578679 MSE: 0.0946986497701175
Time: 7.340760946273804
jannis_MF+OneHot_0.4 jannis_MF+OneHot_0.45
38 38
22 jasmine
X_train shape: (1274, 267) y_train sha

Threshold 0.35000000000000003 MAE: 0.20270924147437638 MSE: 0.07623571905990996
Time: 7.756993055343628
X_train shape: (6931, 267) y_train shape: (6931,)
X_test shape: (266, 267) y_test shape: (266,)
Threshold 0.4 MAE: 0.8824882297282289 MSE: 0.8119341070028937
Time: 3.9885408878326416
X_train shape: (7117, 267) y_train shape: (7117,)
X_test shape: (266, 267) y_test shape: (266,)
Threshold 0.45 MAE: 1.0197043869437348 MSE: 1.086470909809241
Time: 9.008933067321777
27 mfeat-factors
X_train shape: (1203, 267) y_train shape: (1203,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.05 MAE: 0.44246025308226195 MSE: 0.21218930587186205
Time: 1.2964239120483398
X_train shape: (2240, 267) y_train shape: (2240,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.1 MAE: 0.19652806367022305 MSE: 0.05511937962249152
Time: 2.262924909591675
X_train shape: (3822, 267) y_train shape: (3822,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.29065557947

Threshold 0.45 MAE: 0.1926787067816978 MSE: 0.04023234175453334
Time: 3.8593599796295166
32 riccardo
X_train shape: (1771, 267) y_train shape: (1771,)
X_test shape: (60, 267) y_test shape: (60,)
Threshold 0.05 MAE: 0.10457930028907231 MSE: 0.026128053280100712
Time: 2.293492078781128
X_train shape: (2396, 267) y_train shape: (2396,)
X_test shape: (60, 267) y_test shape: (60,)
Threshold 0.1 MAE: 0.10145972247771555 MSE: 0.026131468688646405
Time: 1.7032651901245117
X_train shape: (4374, 267) y_train shape: (4374,)
X_test shape: (60, 267) y_test shape: (60,)
Threshold 0.15000000000000002 MAE: 0.11465436442258108 MSE: 0.028492182062705842
Time: 3.7022080421447754
riccardo_MF+OneHot_0.15 riccardo_MF+OneHot_0.2
20 20
X_train shape: (5432, 267) y_train shape: (5432,)
X_test shape: (60, 267) y_test shape: (60,)
Threshold 0.25 MAE: 0.11155765573512376 MSE: 0.028146528449142654
Time: 4.171404123306274
X_train shape: (6284, 267) y_train shape: (6284,)
X_test shape: (60, 267) y_test shape: (60,)


Threshold 0.1 MAE: 0.672237222100692 MSE: 0.4657619990419586
Time: 2.781930923461914
X_train shape: (4530, 267) y_train shape: (4530,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.15000000000000002 MAE: 0.4960164421012056 MSE: 0.2643801615382767
Time: 4.177887916564941
X_train shape: (5120, 267) y_train shape: (5120,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.2 MAE: 0.11019416932370812 MSE: 0.018573288495365364
Time: 4.896411180496216
X_train shape: (6612, 267) y_train shape: (6612,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.25 MAE: 0.28208074633454466 MSE: 0.09795639518949714
Time: 4.847275733947754
X_train shape: (7383, 267) y_train shape: (7383,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.3 MAE: 0.23381533361042398 MSE: 0.07091254967503785
Time: 5.373402118682861
X_train shape: (7391, 267) y_train shape: (7391,)
X_test shape: (216, 267) y_test shape: (216,)
Threshold 0.35000000000000003 MAE: 0.30098785889185925 MSE: 0.10886